<a href="https://colab.research.google.com/github/damianwgriggs/Schr-dinger-s-Dungeon-Engine/blob/main/Schr%C3%B6dinger's_Dungeon_Engine_But_With_Movement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install cirq

In [1]:
# --- SCHRÖDINGER'S DUNGEON: QUANTUM MAP GENERATOR & PLAYER LOOP ---

# 1. Imports
import cirq
import numpy as np
import time
from IPython.display import clear_output # For clearing output in the Colab cell
# Note: sys is no longer needed since we are using standard input()

# --- CONFIGURATION ---
# Wall and Floor ASCII representations
WALL_CHAR = '██'
FLOOR_CHAR = '  '
PLAYER_CHAR = '☺ '

def generate_quantum_map(grid_size: int = 10) -> list[list[int]]:
    """
    Generates a random dungeon map of size N x N based on quantum collapse.
    Returns a 2D list of integers where 0=Wall and 1=Floor.
    """

    TOTAL_QUBITS = grid_size * grid_size

    # 2. Initialize Quantum Oracle (Circuit Definition)
    circuit = cirq.Circuit()

    for i in range(TOTAL_QUBITS):
        q = cirq.LineQubit(i)

        # Hadamard Gate: Qubit is in 50/50 superposition
        circuit.append(cirq.H(q))

        # Measurement: Forces collapse to 0 (Wall) or 1 (Floor)
        circuit.append(cirq.measure(q, key=f'q_{i}'))

    # 3. Execute the Circuit
    simulator = cirq.Simulator()
    results = simulator.run(circuit, repetitions=1)

    # 4. Process Results and Build the Grid
    int_grid = []
    map_output = "" # For initial text printing

    for row in range(grid_size):
        row_list = []
        row_string = ""
        for col in range(grid_size):
            qubit_index = row * grid_size + col

            # Retrieve the measurement result (0 or 1)
            measurement = results.measurements[f'q_{qubit_index}'][0, 0]

            # Store the integer result (0=Wall, 1=Floor)
            row_list.append(measurement)

            # Prepare the printable map
            row_string += WALL_CHAR if measurement == 0 else FLOOR_CHAR

        int_grid.append(row_list)
        map_output += row_string + "\n"

    # Print the initial generated map
    print(f"\n--- Schrödinger's Dungeon Map ({grid_size}x{grid_size}) ---")
    print(map_output)
    print("-----------------------------------------")
    print(f"Generated {TOTAL_QUBITS} quantum bits (0=Wall, 1=Floor).")

    # Return the 2D integer grid (0s and 1s)
    return int_grid

# ----------------------------------------------------------------------
## 🎮 The Player Loop

def play_dungeon(grid):
    """Allows the user to move a player ('☺') through the generated grid."""

    # Grid Logic: 0 = Wall, 1 = Floor

    # 1. Find a starting spot (First available Floor, where grid value is 1)
    player_x, player_y = -1, -1
    found = False
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            if grid[y][x] == 1:
                player_x, player_y = x, y
                found = True
                break
        if found: break

    if not found:
        print("ERROR: Map is all walls! Cannot start player loop.")
        return

    print("Entering the Quantum Realm... (Use w/a/s/d + Enter to move, 'q' to quit)")
    time.sleep(2)

    while True:
        # Clear output is crucial for 'live' feel in Colab
        clear_output(wait=True)

        # 2. Render the Map with the Player
        print("--- SCHRÖDINGER'S DUNGEON (LIVE) ---")
        for y in range(len(grid)):
            row_display = ""
            for x in range(len(grid[0])):
                if x == player_x and y == player_y:
                    row_display += PLAYER_CHAR
                elif grid[y][x] == 0:    # Wall
                    row_display += WALL_CHAR
                else:                    # Floor
                    row_display += FLOOR_CHAR
            print(row_display)
        print("------------------------------------")
        print(f"Position: {player_x}, {player_y}")

        # 3. Get Interactive Input (Restored for Colab text box functionality)
        move = input("Move (w/a/s/d): ").lower()

        if move == 'q':
            print("Exiting simulation.")
            break

        # 4. Calculate New Position
        new_x, new_y = player_x, player_y

        if move == 'w': new_y -= 1
        elif move == 's': new_y += 1
        elif move == 'a': new_x -= 1
        elif move == 'd': new_x += 1

        # 5. Collision Detection
        grid_height = len(grid)
        grid_width = len(grid[0])

        if (0 <= new_y < grid_height) and (0 <= new_x < grid_width):
            if grid[new_y][new_x] == 0: # Check for Wall (0)
                print("BLOCKED! You hit a Quantum Wall.")
                time.sleep(1)
            else:
                player_x, player_y = new_x, new_y
        else:
            print("VOID! You cannot leave the grid.")
            time.sleep(1)

# ----------------------------------------------------------------------
# --- EXECUTE THE SIMULATION ---

# 1. Generate the map
dungeon_map = generate_quantum_map(grid_size=10)

# 2. Start the interactive player loop
play_dungeon(dungeon_map)

--- SCHRÖDINGER'S DUNGEON (LIVE) ---
██☺ ██████    ██    
        ██        ██
  ████  ████████    
  ████  ██████    ██
    ██████████████  
  ██        ██      
  ██            ██  
  ██  ████████  ████
██            ██    
████  ████  ██████  
------------------------------------
Position: 1, 0


KeyboardInterrupt: Interrupted by user